# CORDEX-CORE urban vs rural vicinity analysis

...

## Load libraries

Here we load the required libraries to execute the notebook. Note that specific code from dedicated scripts (`utils.py`, `urban_areas.py`, ...) is also loaded here.

In [ ]:
%load_ext autoreload
%autoreload 2
    
import cartopy.crs as ccrs
import cf_xarray as cfxr
import dask
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
import xarray as xr
import glob


from icecream import ic
from itertools import product
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely.geometry import Point
from skimage.morphology import dilation, square
from utils import RCM_DICT, MODEL_DICT, YAMLconfig

from urban_areas import (
    load_fix_variables,
    fix_sftuf,
    Urban_vicinity,
    load_variable,
    kelvin2degC,
    load_ucdb_city,
)

from plot_functions import (
    plot_climatology,
    plot_time_series
)

## Input parameters

These parameters define the behaviour of the notebook, by selecting the `city`, `variable`, and some other items which are mostly self-explanatory. _(Explain here those with are not)_

This cell is tagged `parameters`, in order to be managed externally by the [`papermill` package](https://papermill.readthedocs.io/en/latest).

In [ ]:
abbr_city = 'London-EUR-22'

variable = 'tasmin'
urban_var = 'sftimf'
observations = True
domain = 'EUR-22'
model = 'REMO' # REMO/RegCM
scenario = 'evaluation'
root_esgf = '/lustre/gmeteo/DATA/ESGF/REPLICA/DATA/cordex/output/'
root_nextcloud = '/lustre/gmeteo/WORK/DATA/CORDEX-FPS-URB-RCC/nextcloud/CORDEX-CORE-WG/'


lon_city = -0.13
lat_city = 51.50

urban_th = 10
urban_sur_th = 10
orog_diff = 100
sftlf_th = 70
scale = 2
min_city_size = 5 # remove small urban centers (numbero of cells)
lon_lim = 1.5
lat_lim = 1.2

In [ ]:
# Parameters
abbr_city = "Chicago"
lat_city = 41.8
lon_city =  -87.6
domain = "NAM-22"
variable = "tasmin"
urban_var = "sftimf"
model = "REMO"
urban_th = 10
urban_sur_th = 10
orog_diff = 100
sftlf_th = 70
lon_lim = 1.5
lat_lim = 1.2
min_city_size = 3


Create a directory to save the results for this particular city and model

In [ ]:
base_output_dir = f'results'

cities = YAMLconfig('selected_cities.yaml')
city = cities[f"{abbr_city}_{model}_{urban_var}"]['name']
model_str = RCM_DICT[domain][model]
output_dir = f"{base_output_dir}/{abbr_city}-{domain}_{model_str}_{urban_var}"
if domain in ["EUR-11","EUR-22"]:
    output_dir = f"{base_output_dir}/{abbr_city}_{model_str}_{urban_var}"
os.makedirs(output_dir, exist_ok = True)


In [ ]:
domain_resolution = int(domain.split('-')[1])
base_filename = f'{urban_var}_{abbr_city}-{domain}_ECMWF-ERAINT_{scenario}_r1i1p1_{model_str}'
if domain in ["EUR-11","EUR-22"]:
    base_filename = f'{urban_var}_{abbr_city}_ECMWF-ERAINT_{scenario}_r1i1p1_{model_str}'

## Define urban areas and vicinity

### Load fix variables
Load fix (static) variables, such as urban fraction (sfturf), terrain elevation (orog) or land fraction (sftlf). The urban fraction needs some fixes to align the coordinates with the rest of the variables. These are applied in the `fix_sfturf` function.

In [ ]:
model

In [ ]:
file_pattern = f"./interpolation_results/sftimf_*{domain}*{model}*.nc"
file_paths = glob.glob(file_pattern)
ds_sftuf = xr.open_dataset(file_paths[0])
file_pattern = f"./interpolation_results/orog_*{domain}*{model}*.nc"
file_paths = glob.glob(file_pattern)
ds_orog = xr.open_dataset(file_paths[0])
file_pattern = f"./interpolation_results/sftlf_*{domain}*{model}*.nc"
file_paths = glob.glob(file_pattern)
ds_sftlf = xr.open_dataset(file_paths[0])

Add parameters to the function

In [ ]:
URBAN = Urban_vicinity(
    urban_th, 
    urban_sur_th, 
    orog_diff, 
    sftlf_th,
    scale,
    min_city_size,
    urban_var,
    lon_city,
    lat_city,
    lon_lim,
    lat_lim,
    model_str,
    domain)

Crop area around de city

In [ ]:
ds_sftuf = ds_sftuf.sel(lon = slice(lon_city-lon_lim, lon_city+lon_lim ), lat = slice(lat_city-lat_lim, lat_city+lat_lim))
ds_orog = ds_orog.sel(lon = slice(lon_city-lon_lim, lon_city+lon_lim), lat = slice(lat_city-lat_lim, lat_city+lat_lim))
ds_sftlf = ds_sftlf.sel(lon = slice(lon_city-lon_lim, lon_city+lon_lim ), lat = slice(lat_city-lat_lim, lat_city+lat_lim))

Define masks using the parameters above

In [ ]:
sftuf_mask, sftuf_sur_mask, orog_mask, sftlf_mask = URBAN.define_masks(
    ds_sftuf, 
    ds_orog, 
    ds_sftlf
)

### Define rural vicinity areas

We use an iterative algorithm to select the urban vicinity excluding masked areas

In [ ]:
urmask = URBAN.select_urban_vicinity(
    sftuf_mask, 
    orog_mask,
    sftlf_mask,
    sftuf_sur_mask
)

We can plot the masks and the urban vs vicinity areas. The latter are are passed via the `urban_areas` parameter. They are hidden if set to `None`.

In [ ]:
fig = URBAN.plot_fix_variables(ds_sftuf, ds_orog, ds_sftlf,
                                 sftuf_mask, orog_mask, sftlf_mask,
                                 urban_areas = urmask)
fig.savefig(f"{output_dir}/urmask_{base_filename}_fx.pdf", bbox_inches='tight')

Save these masks and urban fraction to NetCDF

In [ ]:
urmask.to_netcdf(f"{output_dir}/urmask_{base_filename}_fx.nc")